In [1]:
from qondense.tapering import tapering
from qondense.cs_vqe import cs_vqe
from qondense.utils.operator_toolkit import exact_gs_energy
import qondense.utils.qonversion_tools as qonvert
from qondense.utils.ordering_heuristics import ordering_heuristics
import json
import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from itertools import combinations

In [2]:
with open('data/molecule_geometries/molecule_data.json') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET'])


In [3]:
# Set molecule parameters
speciesname = 'CH3CN_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]

atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

delete_input = True
delete_output = True
cisd=0
ccsd=0
fci =0 # wouldn't advise turning this on over 32 qubits!

# Run pyscf.
molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
calculated_molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=cisd,run_ccsd=ccsd,run_fci=fci)

In [4]:
oh = ordering_heuristics(calculated_molecule)

------------------------------------------------
Information concerning the full system:
------------------------------------------------
Number of qubits in full problem: 36
The Hartree-Fock state is |111111111111111111111100000000000000>
Hartree-Fock energy   = -130.27154235
Møller–Plesset energy = -130.47373963
------------------------------------------------
Tapering information:
------------------------------------------------
We are able to taper 3 qubits from the Hamiltonian
The symmetry sector is [-1, -1, 1]
The tapered Hartree-Fock state is |111111111111111111100000000000000>
------------------------------------------------
CS-VQE information:
------------------------------------------------
Noncontextual GS energy: -130.271542348818
Symmetry generators:     ['ZIZIZIZIZIZZIZIIZZIZIIZZIZIIZZIZI', 'IZIZIZIZIZZIZIZIZIZIZIZIZIZIZIZIZ', 'IIZIZIZIZIZZIZIIZZIZIIZZIZIIZZIZI', 'IIIZIZIZIZZIZIZIZIZIZIZIZIZIZIZIZ', 'IIIIZIZIZIZZIZIIZZIZIIZZIZIIZZIZI', 'IIIIIZIZIZZIZIZIZIZIZIZIZIZIZIZIZ',

In [5]:
methods = ["a", "b", "c", "d"]#, "e", "f"]
data = {"methods":methods}
for m in methods:
    print('Method: ',m,'\n------------------------------------------')
    data[m] = oh.heuristic_errors(heuristic=m, print_info=True)

data['num_tapered']= oh.n_taper
data['num_qubits'] = oh.n_qubits
data['stabilizers']= oh.generators
data['hf_energy']  = oh.hf_energy
data['mp_energy']  = oh.mp_energy
data['fci_energy'] = oh.fci_energy

with open('data/cs_vqe_results/'+speciesname+'_heuristic_errors.json', 'w') as outfile:
    json.dump(data, outfile)

a
Number of qubits simulated: 1
CS-VQE error w.r.t. HF energy:  0.0000000017
CS-VQE error w.r.t. MP2 energy: 0.2021972782

Number of qubits simulated: 2
CS-VQE error w.r.t. HF energy: -0.0259674188
CS-VQE error w.r.t. MP2 energy: 0.1762298578

Number of qubits simulated: 3
CS-VQE error w.r.t. HF energy: -0.0261528986
CS-VQE error w.r.t. MP2 energy: 0.1760443780

Number of qubits simulated: 4
CS-VQE error w.r.t. HF energy: -0.0439566668
CS-VQE error w.r.t. MP2 energy: 0.1582406097

Number of qubits simulated: 5
CS-VQE error w.r.t. HF energy: -0.0439569067
CS-VQE error w.r.t. MP2 energy: 0.1582403698

Number of qubits simulated: 6
CS-VQE error w.r.t. HF energy: -0.0754794663
CS-VQE error w.r.t. MP2 energy: 0.1267178103

Number of qubits simulated: 7
CS-VQE error w.r.t. HF energy: -0.0777701568
CS-VQE error w.r.t. MP2 energy: 0.1244271197

Number of qubits simulated: 8
CS-VQE error w.r.t. HF energy: -0.0800111992
CS-VQE error w.r.t. MP2 energy: 0.1221860774

Number of qubits simulated: 9
